In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")  # or any other suitable solver, like CPLEX, Gurobi, etc.

   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Deacon\.julia\environments\v1.9\Manifest.toml`


In [44]:
using Distributions
using JuMP
using HiGHS

In [72]:
#function to calculate replacement cost
function ReplacementCost(type, year, age)
    gasCost = [20000,25000,35000,50000,30000] #current cost of gas vehicle of type n
    elecPremium = [10000,5000,15000,20000,20000] #premium paid for electric version of vehicle n
    salvageValue = [5000,6250,8750,20000,7500] #salvage value of gas vehicle of type n
    newElecCost = gasCost[type]+elecPremium[type]*2.71^-(0.2*year) #projected cost of new EV in year y
    depreciationDiscount = (gasCost[type]-salvageValue[type])/20*age^0.8 #older cars were cheaper when they were bought so less depreciation, 0.8 is an arbitrary number
    #discount because a car is old (not selling old gas vehicle but counting new EV as "cheaper" because gas vehicle would need to be replaced soon)
    return newElecCost - depreciationDiscount
 end

ReplacementCost (generic function with 1 method)

In [73]:
#create fleet (will replace code in next block)
function newFleet(sedans, hybrids, mini, vans, trucks) #inputs are number of each vehicle
    s = zeros(sedans, 13)
    for i in 1:sedans
        a = rand(Uniform(1,14)) #create a random age for the vehicle (this is technically the years left until it should be replaced but it works the same)
        for t in 1:13 #for all years
            if t < 8 #if gas cars are still allowed
                if t < 15-a #if the car is younger than it's replacement age
                    s[i,t] = ReplacementCost(1, t, a) + t*fuelCost(1, false) + (13-t)*fuelCost(1, true)
                    #current year average cost of EV sedan with slow depreciation to expected future costs
                else
                    s[i,t] = (ReplacementCost(1, t, a) + t*fuelCost(1, false) + (13-t)*fuelCost(1, true))*1.5 #penalty for not replacing a vehicle by year 15
                end
            else
                s[i,t] = 1000000000 #lower if we want an economic punishment instead of an absolute rule
            end
        end
    end #repeat for all other types of cars
    h = zeros(hybrids, 13)
    for i in 1:hybrids
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < 8 #if gas cars are still allowed
                if t < 15-a #if the car is younger than it's replacement age
                    h[i,t] = ReplacementCost(2, t, a) + t*fuelCost(2, false) + (13-t)*fuelCost(2, true)
                    #current year average cost of EV sedan with slow depreciation to expected future costs
                else
                    h[i,t] = (ReplacementCost(2, t, a) + t*fuelCost(2, false) + (13-t)*fuelCost(2, true))*1.5
                end
            else
                h[i,t] = 1000000000 #lower if we want an economic punishment instead of an absolute rule
            end
        end
    end
    m = zeros(mini, 13)
    for i in 1:mini
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < 15-a
                m[i,t] = ReplacementCost(3, t, a) + t*fuelCost(3, false) + (13-t)*fuelCost(3, true)
            else
                m[i,t] = (ReplacementCost(3, t, a) + t*fuelCost(3, false) + (13-t)*fuelCost(3, true))*1.5
            end      
        end
    end
    v = zeros(vans, 13)
    for i in 1:vans
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < 15-a
                v[i,t] = ReplacementCost(4, t, a) + t*fuelCost(4, false) + (13-t)*fuelCost(4, true)
            else
                v[i,t] = (ReplacementCost(4, t, a) + t*fuelCost(4, false) + (13-t)*fuelCost(4, true))*1.5
            end      
        end
    end
    tr = zeros(trucks, 13)
    for i in 1:trucks
        a = rand(Uniform(1,14))
        for t in 1:13
            if t < 15-a
                tr[i,t] = ReplacementCost(5, t, a) + t*fuelCost(5, false) + (13-t)*fuelCost(5, true)
            else
                tr[i,t] = (ReplacementCost(5, t, a) + t*fuelCost(5, false) + (13-t)*fuelCost(5, true))*1.5
            end      
        end
    end
    fleet = [s,h,m,v,tr]
    return fleet
end

newFleet (generic function with 1 method)

In [76]:
#possible replacement for travelCost function above
function fuelCost(type, electric) #type is a number corresponding to the array index, electric is a boolean
    fleetMiles = [4000,4000,4000,4000,12000] #based on US survey, car and van miles are 1/3 because fleet services says they are only checked out roughly 1/3 the time while trucks are mostly permanently assigned, same order as variables are initialized
    gasCost = [0.18, 0.12, 0.23, 0.32, 0.24] #rough dollars of gas per mile driven, same order as above
    elecCost = [0.05,0.05,0.07,0.08,0.1] #rough dollars of electricity per mile driven, same order
    if electric
        return fleetMiles[type]*elecCost[type]
    end
    return return fleetMiles[type]*gasCost[type]
end

fuelCost (generic function with 1 method)

In [159]:
#generate fleet
sedans = 400
hybrids = 100
minis = 100
vans = 200
trucks = 400

Years = 1:13
Type = 1:5 #sedan, hybrid, minivan, van, truck in that order
S = 1:sedans #sedan size index
H = 1:hybrids
M = 1:minis
V = 1:vans
R = 1:trucks

fleet = newFleet(sedans, hybrids, minis, vans, trucks)
fleetkWh = [1000 1000 1300 1600 6000]
charge_cost2 = [5500 5400 5300 5200 5100 5000 4900 4800 4700 4600 4500 4500 4500]
charge_cost3 = [50000 45000 42000 39000 36000 34000 32000 30000 28000 26500 25000 24000 23000]

println(fleet[5][1:10,:])

[55141.355466558 53859.5184096935 79669.63649128638 79207.93127438871 79285.22725511808 79804.08948605016 80684.69628114361 81861.65527625289 83281.39504861538 84900.02825183656 86681.60103077829 88596.65888874634 90621.07180207079; 62252.391818215205 60970.5547613507 60224.12734584813 59916.32386791635 59967.854521735935 60313.76267569065 60900.833872419615 61685.47320249246 62631.96638406745 63711.0551862149 64898.77037217605 66175.47561082142 67525.0842197044; 60227.27645139315 58945.43939452864 58199.01197902607 57891.20850109429 57942.73915491388 58288.64730886859 58875.71850559756 59660.35783567041 60606.8510172454 61685.93981939285 94310.482508031 96225.54036599906 98249.95327932353; 59211.1534559575 57929.316399093 57182.88898359043 56875.085505658644 56926.61615947823 57272.52431343294 57859.59551016191 58644.23484023476 59590.72802180975 91004.72523593581 92786.29801487752 94701.35587284558 96725.76878617004; 58129.367026386666 56847.52996952216 56101.10255401959 55793.299076

In [160]:
#create model
model = Model(HiGHS.Optimizer)

# Decision variables
@variable(model, e_s[s in S, t in Years] >= 0, Bin)
@variable(model, e_h[h in H, t in Years] >= 0, Bin)
@variable(model, e_m[m in M, t in Years] >= 0, Bin)
@variable(model, e_v[v in V, t in Years] >= 0, Bin)
@variable(model, e_tr[r in R, t in Years] >= 0, Bin)
@variable(model, chargingstation2[t in Years] >= 0, Int)
@variable(model, chargingstation3[t in Years] >= 0, Int)

# Objective function (minimize the total cost)
@objective(model, Min, sum(fleet[1][s, t]*e_s[s, t] for s in S for t in Years)
        + sum(fleet[2][h, t]*e_h[h, t] for h in H for t in Years)
        + sum(fleet[3][m, t]*e_m[m, t] for m in M for t in Years)
        + sum(fleet[4][v, t]*e_v[v, t] for v in V for t in Years)
        + sum(fleet[5][r, t]*e_tr[r, t] for r in R for t in Years)
        + sum(charge_cost2[t]*(chargingstation2[t]) for t in Years)
        + sum(charge_cost3[t]*(chargingstation3[t]) for t in Years))
        
# Constraints
#Each individual vehicle can only be electrified once
for s in S
        @constraint(model, sum(e_s[s, t] for t in Years) == 1)
end
for h in H
        @constraint(model, sum(e_h[h, t] for t in Years) == 1)
end
for m in M
        @constraint(model, sum(e_m[m, t] for t in Years) == 1)
end
for v in V
        @constraint(model, sum(e_v[v, t] for t in Years) == 1)
end
for r in R
        @constraint(model, sum(e_tr[r, t] for t in Years) == 1)
end
#Year 7 electric percentage constraints
@constraint(model, year8sedan, sum(e_s[:,1:8]) >= S[end])
@constraint(model, year8hybrid, sum(e_h[:,1:8]) >= H[end])
@constraint(model, year8mini, sum(e_m[:,1:8]) >= M[end]*.4)
@constraint(model, year8van, sum(e_v[:,1:8]) >= V[end]*.4)
@constraint(model, year8truck, sum(e_tr[:,1:8]) >= R[end]*.4)
#Year 13 final electric percentage constraints (sedans and hybrids do not need this one)
@constraint(model, year13mini, sum(e_m) >= M[end])
@constraint(model, year13van, sum(e_v) >= V[end])
@constraint(model, year13truck, sum(e_tr) >= R[end])
#Charging station constraints
for t in Years
        @constraint(model, 35000*sum(chargingstation2[1:t]) + 240000*sum(chargingstation3[1:t]) >=
                fleetkWh[1]*sum(e_s[:,1:t]) + fleetkWh[2]*sum(e_h[:,1:t]) + fleetkWh[3]*sum(e_m[:,1:t])
                + fleetkWh[4]*sum(e_v[:,1:t]) + fleetkWh[5]*sum(e_tr[:,1:t]))
end

In [161]:
optimize!(model)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1221 rows, 15626 cols, 143682 nonzeros
1218 rows, 8089 cols, 46736 nonzeros

Solving MIP model with:
   1218 rows
   8089 cols (8063 binary, 26 integer, 0 implied int., 0 continuous)
   46736 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.1s
 S       0       0         0   0.00%   0               51887785.88839   100.00%        0      0      0         0     0.2s

15.1% inactive integer columns, restarting
Model after restart has 762 rows, 6590 cols (6563 bin., 27 int., 0 impl., 0 cont.), and 36237 nonzeros

         0       0         0   0.00

In [163]:
@show(value.(chargingstation3))
@show(value.(sum(e_tr[:,5:end])))

value.(chargingstation3) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:13
And data, a 13-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.9999999999999993
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
value.(sum(e_tr[:, 5:end])) = -5.684341886080802e-14


-5.684341886080802e-14